# **Case QuantumFinance - Disciplina NLP - Classificador de chamados**

***Participantes (RM - NOME):***<br>
355729 - ALYSSON MARQUEZELLI S ALMEIDA<br>
356520 - NATANAEL DE OLIVEIRA CASTRO<br>
355741 - VICTOR FERNANDES CARDOSO<br>
356427 - MATEUS DA ROS SCHUMACHER<br>

### **Crie um classificador de chamados aplicando técnicas de PLN**
---

A **QuantumFinance** tem um canal de atendimento via chat e precisar classificar os assuntos dos atendimentos para melhorar as tratativas dos chamados dos clientes. O canal recebe textos abertos dos clientes relatando o problema e/ou dúvida e depois é direcionado para alguma área especialista no assunto para uma melhor tratativa.​

1. Crie um modelo classificador de assuntos aplicando técnicas de PLN, que consiga classificar através de um texto o assunto conforme disponível na base de dados [1] para treinamento e validação do seu modelo.​

  O modelo precisar atingir um score na **métrica F1 Score superior a 75%**. Utilize o dataset [1] para treinar e testar o modelo, separe o dataset em duas amostras (75% para treinamento e 25% para teste com o randon_state igual a 42).​

2. Utilizar ao menos uma aplicação de modelos de GenAI (LLM´s) para criar o modelo classificador com os mesmos critérios do item 1.

Fique à vontade para testar e explorar as técnicas de pré-processamento, abordagens de NLP, algoritmos e bibliotecas, mas explique e justifique as suas decisões durante o desenvolvimento.​

**Composição da nota:​**

**50%** - Demonstrações das aplicações das técnicas de PLN (regras, pré-processamentos, tratamentos, variedade de modelos aplicados, aplicações de GenIA, organização do pipeline, etc.)​

**50%** - Baseado na performance (score) obtida com a amostra de teste no pipeline do modelo campeão (validar com  a Métrica F1 Score). **Separar o pipeline completo do modelo campeão conforme template.​**

O trabalho poderá ser feito em grupo de 2 até 4 pessoas (mesmo grupo do Startup One) e trabalhos iguais serão descontado nota e passível de reprovação.

**[1] = ​https://dados-ml-pln.s3.sa-east-1.amazonaws.com/tickets_reclamacoes_classificados.csv**

**[F1 Score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html)** com average='weighted'

In [2]:
# CARREGANDO O DATA FRAME
import pandas as pd
#df = pd.read_csv('https://dados-ml-pln.s3.sa-east-1.amazonaws.com/tickets_reclamacoes_classificados.csv', delimiter=';')
df = pd.read_csv('tickets_reclamacoes_classificados.csv', delimiter=';')

# Façam o download do arquivo e utilizem localmente durante os testes

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21072 entries, 0 to 21071
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   id_reclamacao         21072 non-null  int64 
 1   data_abertura         21072 non-null  object
 2   categoria             21072 non-null  object
 3   descricao_reclamacao  21072 non-null  object
dtypes: int64(1), object(3)
memory usage: 658.6+ KB


Bom desenvolvimento!

### **Area de desenvolvimento e validações**

Faça aqui as demonstrações das aplicações das técnicas de PLN (regras, pré-processamentos, tratamentos, variedade de modelos aplicados, organização do pipeline, etc.)​

Fique à vontade para testar e explorar as técnicas de pré-processamento, abordagens de NLP, algoritmos e bibliotecas, mas explique e justifique as suas decisões durante o desenvolvimento.​

### **Parte 01 - Uso de ML para Classificação de Textos**

#### Exploração & Pré-processamento dos Textos

Além de explorar os dados, aqui aplicamos técnicas clássicas de normalização e limpeza de texto, como:

- Lowercasing
- Remoção de pontuação, números e padrões via regex (ex: "xx/xx/xxxx", "xxxx", valores monetários)
- Remoção de acentos
- Tokenização
- Remoção de stopwords (incluindo personalização)
- Filtro de palavras curtas

O objetivo é preparar os textos para vetorização e modelagem.

In [4]:
df.isnull().sum()

id_reclamacao           0
data_abertura           0
categoria               0
descricao_reclamacao    0
dtype: int64

In [5]:
df['categoria'].value_counts()
df['categoria'].value_counts(normalize=True) * 100  # Em %


categoria
Serviços de conta bancária             24.492217
Cartão de crédito / Cartão pré-pago    23.756644
Roubo / Relatório de disputa           22.883447
Hipotecas / Empréstimos                18.270691
Outros                                 10.597001
Name: proportion, dtype: float64

In [6]:
# verificanr tamanho dos textos 

df['tamanho_texto'] = df['descricao_reclamacao'].apply(lambda x: len(str(x).split()))
df['tamanho_texto'].describe()
df_alt = df


In [7]:
# explorando alguns textos por categoria
for categoria in df['categoria'].unique():
    exemplo = df[df['categoria'] == categoria]['descricao_reclamacao'].iloc[0]
    print(f"\n >>> Categoria: {categoria}\nTexto: {exemplo}")



 >>> Categoria: Hipotecas / Empréstimos
Texto: Bom dia, meu nome é xxxx xxxx e agradeço se você puder me ajudar a acabar com os serviços de membro do cartão bancário.
Em 2018, escrevi para Chase solicitar verificação da dívida e o que eles me enviaram uma declaração que não é aceitável. Estou pedindo ao banco que valide a dívida. Em vez disso, recebi e -mails todos os meses, tentando coletar uma dívida.
Tenho o direito de conhecer essas informações como consumidor.

Conta do Chase # xxxx xxxx xxxx xxxx Obrigado antecipadamente pela sua ajuda.

 >>> Categoria: Cartão de crédito / Cartão pré-pago
Texto: Atualizei meu cartão xxxx xxxx em xx/xx/2018 e fui informado pelo agente que fez a atualização que minha data de aniversário não mudaria. Ele virou o agente me dando as informações erradas para atualizar a conta. Xxxx alterou minha data de aniversário de xx/xx/xxxx para xx/xx/xxxx sem meu consentimento! XXXX tem a gravação do agente que me enganou.

 >>> Categoria: Serviços de conta banc

#### Pré-processamento

In [8]:
!pip install nltk unicode

Defaulting to user installation because normal site-packages is not writeable


In [9]:
import nltk
nltk.download('punkt_tab') # obtive problemas com o punkt -> essa versao punkt_tab é obsoleta pela documentação
nltk.download('stopwords')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\alyss\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\alyss\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [10]:
import string
import nltk
import re
from nltk.tokenize import word_tokenize

# Lista de stopwords em português
stopwords = nltk.corpus.stopwords.words('portuguese')

# Função para remover pontuação
def remove_punctuation(text):
    punctuations = string.punctuation
    table = str.maketrans({key: " " for key in punctuations})
    text = text.translate(table)
    return text

# Normalização e tokenização
def norm_tokenize(text):
    text = str(text).lower()
    
    # Limpeza com regex 
    text = re.sub(r'xx+/xx+/xxxx?', ' ', text)  # datas como xx/xx/xxxx
    text = re.sub(r'\$?\s?\d+(?:,\d+)?', ' ', text)  # valores monetários
    text = re.sub(r'(?i)\b[x]{2,}\b', ' ', text)  # palavras com xxxx, xx etc
    text = re.sub(r'\d+', ' ', text)  # remove números restantes
    text = re.sub(r'\s+', ' ', text)  # espaços duplicados

    # Remover pontuação
    text = remove_punctuation(text)

    # Tokenização
    text = word_tokenize(text)

    # Remoção de stopwords e palavras curtas
    text = [x for x in text if x not in stopwords]
    text = [y for y in text if len(y) > 2]

    return " ".join(text)

# Aplicando o pré-processamento
df_alt['texto_processado'] = df_alt['descricao_reclamacao'].apply(norm_tokenize)

# Mostrando antes e depois para alguns exemplos
df_alt[['descricao_reclamacao', 'texto_processado']].head()


,descricao_reclamacao,texto_processado
0,"Bom dia, meu nome é xxxx xxxx e agradeço se vo...",bom dia nome agradeço puder ajudar acabar serv...
1,Atualizei meu cartão xxxx xxxx em xx/xx/2018 e...,atualizei cartão informado agente fez atualiza...
2,O cartão Chase foi relatado em xx/xx/2019. No ...,cartão chase relatado entanto pedido fraudulen...
3,"Em xx/xx/2018, enquanto tentava reservar um ti...",enquanto tentava reservar ticket deparei ofert...
4,"Meu neto me dê cheque por {$ 1600,00} Eu depos...",neto cheque depositei conta chase fundo limpo ...


##### Teste de Vetorizadores e Modelos de Classificação

Nesta etapa, comparamos diferentes combinações de vetorizadores e algoritmos de classificação para identificar o pipeline com melhor desempenho.

- **TF-IDF (1-2):** Considera unigramas e bigramas com normalização TF-IDF
- **TF-IDF (1-3):** Considera unigramas, bigramas e trigramas (mais contexto)
- **BoW (1-2):** Modelo tradicional de contagem com unigramas e bigramas

Modelos de classificação testados:
- **Naive Bayes (MultinomialNB):** 
- **Logistic Regression:** 
- **LinearSVC:** 
- **Random Forest:** 

Parâmetros:
- Todos os vetores foram limitados a **5.000 features** (usando `max_features=5000`)
- Dados separados em **75% treino / 25% teste** com `random_state=42` e `stratify=y`
- Métrica usada: **F1 Score (weighted)** — ideal para bases com classes desbalanceadas

A seguir, apresentamos uma tabela com os resultados ordenados pelo maior F1 Score.




In [11]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
import pandas as pd

X = df_alt['texto_processado']
y = df_alt['categoria']

vetorizadores = {
    'TF-IDF (1-2)': TfidfVectorizer(ngram_range=(1, 2), max_features=5000),
    'TF-IDF (1-3)': TfidfVectorizer(ngram_range=(1, 3), max_features=5000),
    'BoW (1-2)': CountVectorizer(ngram_range=(1, 2), max_features=5000),
}

modelos = {
    'Naive Bayes': MultinomialNB(),
    'Logistic Regression': LogisticRegression(max_iter=1000),
    'LinearSVC': LinearSVC(),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42)
}

resultados = []

for nome_vet, vet in vetorizadores.items():
    X_vet = vet.fit_transform(X)
    X_train, X_test, y_train, y_test = train_test_split(
        X_vet, y, test_size=0.25, random_state=42, stratify=y
    )

    for nome_modelo, modelo in modelos.items():
        modelo.fit(X_train, y_train)
        y_pred = modelo.predict(X_test)
        f1 = f1_score(y_test, y_pred, average='weighted')

        resultados.append({
            'Vetorizador': nome_vet,
            'Modelo': nome_modelo,
            'F1 Score': round(f1, 4)
        })

# Exibir resultados como DataFrame
df_resultados = pd.DataFrame(resultados)
print(df_resultados.sort_values(by='F1 Score', ascending=False))


C:\Users\alyss\AppData\Roaming\Python\Python312\site-packages\sklearn\svm\_base.py:1235: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


     Vetorizador               Modelo  F1 Score
1   TF-IDF (1-2)  Logistic Regression    0.9059
5   TF-IDF (1-3)  Logistic Regression    0.9045
9      BoW (1-2)  Logistic Regression    0.9031
6   TF-IDF (1-3)            LinearSVC    0.8986
2   TF-IDF (1-2)            LinearSVC    0.8971
10     BoW (1-2)            LinearSVC    0.8817
7   TF-IDF (1-3)        Random Forest    0.8503
3   TF-IDF (1-2)        Random Forest    0.8491
11     BoW (1-2)        Random Forest    0.8385
8      BoW (1-2)          Naive Bayes    0.8250
0   TF-IDF (1-2)          Naive Bayes    0.8087
4   TF-IDF (1-3)          Naive Bayes    0.8073


In [12]:
# Visualizar tabela de resutlados
df_resultados = pd.DataFrame(resultados).sort_values(by='F1 Score', ascending=False).reset_index(drop=True)

# Estilizar a visualização
styled = df_resultados.style.set_caption("Comparativo de Vetorizadores e Modelos") \
    .background_gradient(subset=['F1 Score'], cmap='Greens') \
    .format({'F1 Score': '{:.4f}'}) \
    .hide(axis='index') \
    .set_table_styles([{
        'selector': 'caption',
        'props': [('color', '#333'), ('font-size', '16px'), ('font-weight', 'bold')]
    }])

styled


Vetorizador,Modelo,F1 Score
TF-IDF (1-2),Logistic Regression,0.9059
TF-IDF (1-3),Logistic Regression,0.9045
BoW (1-2),Logistic Regression,0.9031
TF-IDF (1-3),LinearSVC,0.8986
TF-IDF (1-2),LinearSVC,0.8971
BoW (1-2),LinearSVC,0.8817
TF-IDF (1-3),Random Forest,0.8503
TF-IDF (1-2),Random Forest,0.8491
BoW (1-2),Random Forest,0.8385
BoW (1-2),Naive Bayes,0.8250


###### Nosso modelo campeão escolhido foi Regressão Logísica com 0.90 de F1 utilizando o TF-IDF(1-2) como vetorizador

### **Parte 2 — Classificação de Texto com GenAI (Embeddings com Transformers)**

Nesta etapa, vamos aplicar técnicas de PLN com modelos baseados em Transformers para classificar os chamados dos clientes da QuantumFinance.



#### 2.1 Setup Inicial

```python
# Etapa 1 de 10
# Importa bibliotecas essenciais e define o dispositivo (GPU ou CPU) 

## Parte 2: Modelagem com LLM (BERT)

```

In [13]:
# Limpa todas as variáveis do notebook para tentar algum isolamento
from IPython import get_ipython
get_ipython().magic('reset -sf')


C:\Users\alyss\AppData\Local\Temp\ipykernel_19744\2241865917.py:3: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  get_ipython().magic('reset -sf')


In [14]:
import torch
import pandas as pd
import numpy as np
import re
import string
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score
from sklearn import preprocessing
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm

In [15]:
def set_seed(seed=42): #tive problemas com random_state
    import random
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

# Verifica o dispositivo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Executando em:", device)

Executando em: cuda


In [16]:
# Parâmetros ajustáveis para facilitar tuning posterior

MAX_LENGTH = 256       # Tamanho máximo de tokens por entrada
BATCH_SIZE = 32        # Tamanho do batch para treino
LEARNING_RATE = 5e-5   # Taxa de aprendizado
EPOCHS = 5             # Número de épocas de treinamento
DEV_MODE = False       # Usa apenas uma amostra do dataset para acelerar testes

#### 2. Carregamento dos Dados

```python
# Etapa 2 de 10


In [17]:
caminho_arquivo = 'tickets_reclamacoes_classificados.csv'
df = pd.read_csv(caminho_arquivo, sep=';')

# Ativa modo de desenvolvimento com amostragem reduzida
if DEV_MODE:
    df = df.sample(frac=0.1, random_state=42).reset_index(drop=True)
    print(f"[MODO DEV ATIVADO] Usando {len(df)} amostras")

print("Amostra dos dados:")
display(df.head())

print("Colunas disponíveis:", df.columns.tolist())
print("\nDistribuição das categorias:")
print(df['categoria'].value_counts())

Amostra dos dados:


,id_reclamacao,data_abertura,categoria,descricao_reclamacao
0,3229299,2019-05-01T12:00:00-05:00,Hipotecas / Empréstimos,"Bom dia, meu nome é xxxx xxxx e agradeço se vo..."
1,3199379,2019-04-02T12:00:00-05:00,Cartão de crédito / Cartão pré-pago,Atualizei meu cartão xxxx xxxx em xx/xx/2018 e...
2,3233499,2019-05-06T12:00:00-05:00,Cartão de crédito / Cartão pré-pago,O cartão Chase foi relatado em xx/xx/2019. No ...
3,3180294,2019-03-14T12:00:00-05:00,Cartão de crédito / Cartão pré-pago,"Em xx/xx/2018, enquanto tentava reservar um ti..."
4,3224980,2019-04-27T12:00:00-05:00,Serviços de conta bancária,"Meu neto me dê cheque por {$ 1600,00} Eu depos..."


Colunas disponíveis: ['id_reclamacao', 'data_abertura', 'categoria', 'descricao_reclamacao']

Distribuição das categorias:
categoria
Serviços de conta bancária             5161
Cartão de crédito / Cartão pré-pago    5006
Roubo / Relatório de disputa           4822
Hipotecas / Empréstimos                3850
Outros                                 2233
Name: count, dtype: int64


#### Etapa 3 de 10
- Normaliza os textos 

In [18]:
institution_names = ['chase', 'bank', 'jp', 'gm', 'financial', 'jpmcb']

def remove_punctuation(text):
    table = str.maketrans('', '', string.punctuation)
    return text.translate(table)

def normalize_text(text):
    text = text.lower()
    text = re.sub(r'\d+|/', '', text)
    text = re.sub(r'\bx\b|\w*xx+\w*', '', text)
    text = remove_punctuation(text)
    text = re.sub(r'\s+', ' ', text).strip()

    for institution in institution_names:
        text = re.sub(r'\b' + re.escape(institution) + r'\b', '[INST]', text)

    return text

# Cria nova coluna com texto normalizado
df['texto_limpo'] = df['descricao_reclamacao'].apply(normalize_text)

# Visualiza resultado
print("Pré-visualização das colunas originais e normalizadas:")
df[['descricao_reclamacao', 'texto_limpo']].head()

Pré-visualização das colunas originais e normalizadas:


,descricao_reclamacao,texto_limpo
0,"Bom dia, meu nome é xxxx xxxx e agradeço se vo...",bom dia meu nome é e agradeço se você puder me...
1,Atualizei meu cartão xxxx xxxx em xx/xx/2018 e...,atualizei meu cartão em e fui informado pelo a...
2,O cartão Chase foi relatado em xx/xx/2019. No ...,o cartão [INST] foi relatado em no entanto o p...
3,"Em xx/xx/2018, enquanto tentava reservar um ti...",em enquanto tentava reservar um ticket me depa...
4,"Meu neto me dê cheque por {$ 1600,00} Eu depos...",meu neto me dê cheque por eu depositei o na mi...


#### 4. Tokenização com BERT
```python
# Etapa 4 de 10
# Tokeniza os textos normalizados usando BERT e armazena input_ids e attention_mask

In [19]:
tokenizer = BertTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased')

# Tokenização e vetorização dos textos normalizados
def tokenize_bert(text):
    tokens = tokenizer(
        text,
        padding='max_length',
        truncation=True,
        max_length=MAX_LENGTH,
        return_tensors='pt'
    )
    return tokens

# Aplica tokenização e extrai os tensores como listas
encoded = df['texto_limpo'].apply(tokenize_bert)
df['input_ids'] = encoded.apply(lambda x: x['input_ids'].squeeze().tolist())
df['attention_mask'] = encoded.apply(lambda x: x['attention_mask'].squeeze().tolist())

# Visualiza colunas com embeddings
print("Pré-visualização dos textos tokenizados:")
df[['texto_limpo', 'input_ids', 'attention_mask']].head()

Pré-visualização dos textos tokenizados:


,texto_limpo,input_ids,attention_mask
0,bom dia meu nome é e agradeço se você puder me...,"[101, 4062, 644, 7343, 655, 253, 122, 13406, 5...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,atualizei meu cartão em e fui informado pelo a...,"[101, 2233, 15134, 22283, 7343, 12807, 173, 12...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,o cartão [INST] foi relatado em no entanto o p...,"[101, 146, 12807, 164, 13760, 11846, 166, 262,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,em enquanto tentava reservar um ticket me depa...,"[101, 173, 1139, 17366, 6105, 22282, 222, 964,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,meu neto me dê cheque por eu depositei o na mi...,"[101, 7343, 10588, 311, 121, 22325, 765, 455, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


#### 5. Rótulos

```python
# Etapa 5 de 10
# Codifica os rótulos da coluna 'categoria' em valores numéricos

In [20]:
label_encoder = preprocessing.LabelEncoder()
df['label'] = label_encoder.fit_transform(df['categoria'])

# Visualiza a correspondência
print("Categorias codificadas:")
display(pd.DataFrame({
    'categoria': label_encoder.classes_,
    'label': list(range(len(label_encoder.classes_)))
}))

Categorias codificadas:


,categoria,label
0,Cartão de crédito / Cartão pré-pago,0
1,Hipotecas / Empréstimos,1
2,Outros,2
3,Roubo / Relatório de disputa,3
4,Serviços de conta bancária,4


#### 6. Tensores e Split Treino/Teste

```python
# Etapa 6 de 10
# Cria os tensores (input_ids, attention_mask, label) e divide os dados (75/25 com random_state=42)

In [21]:
# Converte listas para tensores
input_ids_tensor = torch.tensor(df['input_ids'].tolist())
attention_mask_tensor = torch.tensor(df['attention_mask'].tolist())
labels_tensor = torch.tensor(df['label'].tolist())

# Realiza o split dos índices para manter os dados organizados
train_idx, test_idx = train_test_split(
    range(len(df)),
    test_size=0.25,
    random_state=42,
    stratify=df['label']  # garante distribuição proporcional
)

# Cria conjuntos de treino e teste
train_dataset = TensorDataset(
    input_ids_tensor[train_idx],
    attention_mask_tensor[train_idx],
    labels_tensor[train_idx]
)

test_dataset = TensorDataset(
    input_ids_tensor[test_idx],
    attention_mask_tensor[test_idx],
    labels_tensor[test_idx]
)

print(f"Tamanho do treino: {len(train_dataset)}")
print(f"Tamanho do teste: {len(test_dataset)}")


Tamanho do treino: 15804
Tamanho do teste: 5268


#### 7. Construção e Treinamento do Modelo BERT

```python
# Etapa 7 de 10
# Instancia o modelo BERT para classificação e realiza o treinamento

In [22]:
num_labels = len(label_encoder.classes_)
model = BertForSequenceClassification.from_pretrained(
    'neuralmind/bert-base-portuguese-cased',
    num_labels=num_labels
)
model.to(device)

# Otimizador e parâmetros
optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)
batch_size = BATCH_SIZE
epochs = EPOCHS

# Loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Treinamento
model.train()
for epoch in range(epochs):
    print(f"\nÉpoca {epoch + 1}/{epochs}")
    for batch in tqdm(train_loader):
        input_ids, attention_mask, labels = [x.to(device) for x in batch]

        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )

        loss = outputs.loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Loss final da época {epoch + 1}: {loss.item():.4f}")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at neuralmind/bert-base-portuguese-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Época 1/5


100%|██████████| 494/494 [05:17<00:00,  1.56it/s]


Loss final da época 1: 0.2078

Época 2/5


100%|██████████| 494/494 [05:16<00:00,  1.56it/s]


...

Loss final da época 3: 0.3362

Época 4/5


100%|██████████| 494/494 [05:16<00:00,  1.56it/s]


Loss final da época 4: 0.1204

Época 5/5


100%|██████████| 494/494 [05:16<00:00,  1.56it/s]


Loss final da época 5: 0.0700


#### 8. Avaliação do Modelo

```python
# Etapa 8 de 10
# Avalia o modelo no conjunto de teste usando F1 Score e classification report

In [23]:
from sklearn.metrics import classification_report, f1_score

model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for batch in DataLoader(test_dataset, batch_size=BATCH_SIZE):
        input_ids, attention_mask, labels = [x.to(device) for x in batch]
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        preds = torch.argmax(outputs.logits, dim=1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Relatório de classificação
print(classification_report(all_labels, all_preds, target_names=label_encoder.classes_))

# F1 Score Macro
f1 = f1_score(all_labels, all_preds, average='macro')
print(f"F1 Score (macro): {f1:.4f}")

                                     precision    recall  f1-score   support

Cartão de crédito / Cartão pré-pago       0.85      0.90      0.88      1252
            Hipotecas / Empréstimos       0.88      0.90      0.89       962
                             Outros       0.74      0.84      0.78       558
       Roubo / Relatório de disputa       0.91      0.79      0.84      1206
         Serviços de conta bancária       0.89      0.89      0.89      1290

                           accuracy                           0.87      5268
                          macro avg       0.86      0.86      0.86      5268
                       weighted avg       0.87      0.87      0.87      5268

F1 Score (macro): 0.8580


#### 9. Salvamento do Modelo

```python
# Etapa 9 de 10
# Salva o modelo treinado, o tokenizer e o codificador de rótulos (label encoder)

In [24]:
import joblib

# Cria pasta se não existir
import os
os.makedirs("modelo_quantumfinance", exist_ok=True)

# Salva modelo e tokenizer
model.save_pretrained("modelo_quantumfinance")
tokenizer.save_pretrained("modelo_quantumfinance")

# Salva o label encoder
joblib.dump(label_encoder, "modelo_quantumfinance/label_encoder.pkl")

print("Modelo, tokenizer e label encoder salvos em 'modelo_quantumfinance/'")

Modelo, tokenizer e label encoder salvos em 'modelo_quantumfinance/'


#### 10. Predição com Novos Textos 

```python
# Etapa 10 de 10
# Usa o modelo treinado para prever a categoria de um novo texto

In [25]:
def prever_texto(texto):
    texto_limpo = normalize_text(texto)
    tokens = tokenizer(texto_limpo, return_tensors='pt', truncation=True, padding='max_length', max_length=MAX_LENGTH)
    tokens = {k: v.to(device) for k, v in tokens.items()}
    
    with torch.no_grad():
        output = model(**tokens)
        pred = torch.argmax(output.logits, dim=1).cpu().item()

    return label_encoder.inverse_transform([pred])[0]



In [26]:
casos_teste = {
    "caso_cartao_1": "Recebi uma cobrança indevida no meu cartão de crédito e não consigo cancelar.",
    "caso_cartao_2": "Preciso desbloquear meu cartão pré-pago que foi bloqueado sem explicação.",
    "caso_emprestimo": "Estou com problemas para renegociar meu empréstimo estudantil.",
    "caso_hipoteca": "O banco está me cobrando taxas abusivas na minha hipoteca.",
    "caso_conta": "Minha conta foi encerrada sem aviso e perdi o acesso ao meu saldo.",
    "caso_roubo_1": "Alguém fez compras no meu nome, quero reportar uma fraude.",
    "caso_roubo_2": "Tem transações estranhas no meu extrato, acho que fui vítima de golpe.",
    "caso_outros_1": "Quero atualizar meu endereço de correspondência.",
    "caso_outros_2": "Estou tentando mudar minha senha, mas o sistema não deixa.",
}

for nome, texto in casos_teste.items():
    categoria = prever_texto(texto)
    print(f"{nome}: {categoria}")

caso_cartao_1: Roubo / Relatório de disputa
caso_cartao_2: Cartão de crédito / Cartão pré-pago
caso_emprestimo: Hipotecas / Empréstimos
caso_hipoteca: Hipotecas / Empréstimos
caso_conta: Serviços de conta bancária
caso_roubo_1: Roubo / Relatório de disputa
caso_roubo_2: Roubo / Relatório de disputa
caso_outros_1: Hipotecas / Empréstimos
caso_outros_2: Roubo / Relatório de disputa


### **Validação do professor**

Consolidar apenas os scripts do seu **modelo campeão**, desde o carregamento do dataframe, separação das amostras, tratamentos utilizados (funções, limpezas, etc.), criação dos objetos de vetorização dos textos e modelo treinado e outras implementações utilizadas no processo de desenvolvimento do modelo.

O modelo precisar atingir um score na métrica F1 Score superior a 75%.

**Atenção:** **Implemente aqui apenas os scripts que fazem parte do modelo campeão.**


In [27]:
# PIPILINE FINAL: TF-IDF (1-2) + Regressão Logística

# Imports
import pandas as pd
import string
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

# Setup NLTK
nltk.download('punkt')
#nltk.download('punkt_tab') # descomentar em caso de problemas 
nltk.download('stopwords')

# dataset
#df = pd.read_csv('https://dados-ml-pln.s3.sa-east-1.amazonaws.com/tickets_reclamacoes_classificados.csv', delimiter=';') 
df = pd.read_csv('tickets_reclamacoes_classificados.csv', sep=';')

# Funções de pré-processamento
stopwords = set(stopwords.words('portuguese'))

def remove_punctuation(text):
    table = str.maketrans({key: " " for key in string.punctuation})
    return text.translate(table)

def norm_tokenize(text):
    text = str(text).lower()
    text = re.sub(r'xx+/xx+/xxxx?', ' ', text)
    text = re.sub(r'\$?\s?\d+(?:,\d+)?', ' ', text)
    text = re.sub(r'(?i)\b[x]{2,}\b', ' ', text)
    text = re.sub(r'\d+', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    text = remove_punctuation(text)
    tokens = text.split()
    tokens = [t for t in tokens if t not in stopwords and len(t) > 2]
    return " ".join(tokens)

# Aplicando pré-processamento
df['texto_processado'] = df['descricao_reclamacao'].apply(norm_tokenize)

# Separação dos dados
X = df['texto_processado']
y = df['categoria']
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42, stratify=y
)

# Vetorização com TF-IDF (1-2)
vectorizer = TfidfVectorizer(ngram_range=(1, 2), max_features=5000)
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

# Modelo campeão: Regressão Logística
modelo = LogisticRegression(max_iter=1000)
modelo.fit(X_train_vec, y_train)

# Avaliação com F1 Score
y_pred = modelo.predict(X_test_vec)
f1 = f1_score(y_test, y_pred, average='weighted')
print(f'´ [ x ] F1 Score do modelo campeão (TF-IDF + Regressão Logística): {f1:.4f}')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\alyss\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\alyss\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


´ [ x ] F1 Score do modelo campeão (TF-IDF + Regressão Logística): 0.9047


## **Conclusão**

Este projeto atingiu com sucesso o objetivo de construir um classificador de assuntos para a QuantumFinance com F1 Score superior a 75%.

- A abordagem com **TF-IDF + Regressão Logística** apresentou desempenho superior (~90%), sendo ideal para ambientes leves e rápidos.
- O modelo BERT fine-tunado atingiu ~86% de F1, mostrando bom entendimento contextual, mesmo com menos dados.

* Considerações finais:
- Para produção, o modelo clássico pode ser preferido pela leveza.
- Para maior robustez e interpretação semântica, o BERT fine-tunado é altamente promissor.


